load the clean dataset


In [1]:
import pandas as pd

import numpy as np
import os

In [2]:
cleaned_data_path = '/home/samrawit/creditrust-RAG/data/filtered_and_cleaned_complaints.csv'
df_cleaned = pd.read_csv(cleaned_data_path, encoding='utf-8')

Load the libraries

In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import faiss
import numpy as np
import os

In [4]:
df_cleaned.columns


Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID',
       'narrative_length', 'cleaned_narrative'],
      dtype='object')

In [6]:
df_cleaned_focus = df_cleaned[['Complaint ID', 'Product', 'cleaned_narrative']].copy()


In [7]:
assert 'cleaned_narrative' in df_cleaned_focus.columns, "Missing 'cleaned_narrative'"
assert 'Complaint ID' in df_cleaned_focus.columns, "Missing 'Complaint ID'"
assert 'Product' in df_cleaned_focus.columns, "Missing 'Product'"

In [12]:
df_cleaned_focus.columns

Index(['Complaint ID', 'Product', 'cleaned_narrative'], dtype='object')

In [13]:
df_cleaned_focus.head()

,Complaint ID,Product,cleaned_narrative
0,14069121,Credit card,card wa opened name fraudster received notice ...
1,14047085,Credit card,dear cfpb secured credit card citibank changed...
2,14040217,Credit card,citi reward card credit balance issued wa rece...
3,13968411,Credit card,bi writing dispute following charge citi credi...
4,13965746,Credit card,although account deemed closed continued make ...


Text chunking

Load embedding model

In [17]:
embedding_model_name = 'all-MiniLM-L6-v2'
print(f"Loading embedding model: {embedding_model_name}...")

try:
    model = SentenceTransformer(embedding_model_name)
    print("Embedding model loaded successfully.")
except Exception as e:
    print(f"Error loading embedding model: {e}")
    raise

# Test the model with a sample text
sample_text = "This is a test sentence for the embedding model."
sample_embedding = model.encode(sample_text)
print(f"Sample text embedding shape: {sample_embedding.shape}")


Loading embedding model: all-MiniLM-L6-v2...
Embedding model loaded successfully.
Sample text embedding shape: (384,)


Create FAISS index

In [ ]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  

print("FAISS index created and embeddings added.")

In [19]:
list_of_chunk_contents = df_cleaned_focus['cleaned_narrative'].tolist()
embeddings_np = model.encode(list_of_chunk_contents, show_progress_bar=True, convert_to_numpy=True)
print(f"Shape of generated embeddings: {embeddings_np.shape}")

Batches:   0%|          | 0/2623 [00:00<?, ?it/s]

Shape of generated embeddings: (83910, 384)


In [20]:
faiss_index = faiss.IndexFlatIP(embeddings_np.shape[1])
faiss_index.add(embeddings_np)
print(f"Total number of vectors in FAISS index: {faiss_index.ntotal}")

Total number of vectors in FAISS index: 83910


Save the FAISS Index and Metadata 

In [25]:

vector_store_dir = '../vector_store/'
os.makedirs(vector_store_dir, exist_ok=True)

In [26]:
faiss_index_path = os.path.join(vector_store_dir, 'faiss_index.bin')
metadata_path = os.path.join(vector_store_dir, 'metadata.json')

Save vector index and metadata

In [27]:
faiss.write_index(faiss_index, faiss_index_path)
print("FAISS index saved.")

# Prepare metadata
metadata = df_cleaned_focus[['Complaint ID', 'Product']].to_dict(orient='records')

# Save metadata
import json
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=4)
print("Metadata saved.")



FAISS index saved.
Metadata saved.
